In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Path

In [ ]:
!ls

In [ ]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


# Requirements

In [ ]:
!pip install ogb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=6f64ea258e1992201d2e6ba8808d74a317f9b650504039765a424d7d5f131b53
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 72.6 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.datasets import OGB_MAG
from torch_geometric.data import Data, HeteroData
from torch_geometric.nn import SAGEConv, to_hetero, GATConv, Linear
import torch_geometric.transforms as T
from torch import nn

# Raw Data

In [ ]:
from src.features import get_relations, get_entities, get_relation_types, get_associations

## Entities

In [ ]:
raw_entities = get_entities()
raw_entities.head()

,type,name,id
0,Compound,CHEBI:74510,0
1,Gene,10584,1
2,Metabolite,HMDB0075603,2
3,Gene,79134,3
4,Disease,DOID:12306,4


## Relations

In [ ]:
raw_relations = get_relations()
raw_relations.head()

,head,relation,tail
0,37448,17,41929
1,41929,17,37448
2,45065,17,41929
3,41929,17,45065
4,50858,17,41929


## Relation Types

In [ ]:
raw_relation_types = get_relation_types()
raw_relation_types.head()

,type,name,id
0,VMH:metabolite-disease:Decreased,metabolite:disease,0
1,GNBR:Ud,Gene:Disease,1
2,GNBR:L,Gene:Disease,2
3,GNBR:Pa,Compound:Disease,3
4,VMH:metabolite-disease:Increased,metabolite:disease,4


## Associations

In [ ]:
raw_associations = get_associations()
raw_associations.tail()

,disease,microbe,increased
893,64642,53920,0
894,25026,60601,0
895,25026,44316,0
896,31069,60226,0
897,64642,4251,0


In [ ]:
y = torch.tensor(raw_associations['increased'].tolist(), dtype=torch.float32).reshape(-1, 1)
y.shape

torch.Size([898, 1])

# Heterogeneous Data

In [ ]:
data = HeteroData()

In [ ]:
entity_types = list(set(raw_entities['type'].tolist()))

In [ ]:
for e in entity_types:
    data[e].x = torch.tensor(raw_entities.loc[raw_entities['type'] == e]['id'].tolist()).reshape(-1, 1)

In [ ]:
data

HeteroData(
  Disease={ x=[5645, 1] },
  Symptom={ x=[415, 1] },
  Anatomy={ x=[398, 1] },
  Metabolite={ x=[23000, 1] },
  Microbe={ x=[5179, 1] },
  Division={ x=[111, 1] },
  Organ={ x=[55, 1] },
  Compound={ x=[9572, 1] },
  Gene={ x=[22536, 1] }
)

In [ ]:
relation_types = raw_relation_types['type'].tolist()
relation_ids = raw_relation_types['id'].tolist()

In [ ]:
for i in range(len(relation_types)):
    a = raw_relations.loc[raw_relations['relation'] == relation_ids[i]]['head'].tolist()
    b = raw_relations.loc[raw_relations['relation'] == relation_ids[i]]['tail'].tolist()
    head_type = raw_entities.loc[raw_entities['id'] == a[0]]['type'].item()
    tail_type = raw_entities.loc[raw_entities['id'] == b[0]]['type'].item()
    data[head_type, relation_types[i], tail_type].edge_index = torch.tensor([a, b])

In [ ]:
data

HeteroData(
  Disease={ x=[5645, 1] },
  Symptom={ x=[415, 1] },
  Anatomy={ x=[398, 1] },
  Metabolite={ x=[23000, 1] },
  Microbe={ x=[5179, 1] },
  Division={ x=[111, 1] },
  Organ={ x=[55, 1] },
  Compound={ x=[9572, 1] },
  Gene={ x=[22536, 1] },
  (Metabolite, VMH:metabolite-disease:Decreased, Disease)={ edge_index=[2, 38] },
  (Gene, GNBR:Ud, Disease)={ edge_index=[2, 814] },
  (Gene, GNBR:L, Disease)={ edge_index=[2, 96768] },
  (Compound, GNBR:Pa, Disease)={ edge_index=[2, 5238] },
  (Metabolite, VMH:metabolite-disease:Increased, Disease)={ edge_index=[2, 192] },
  (Disease, Hetionet:DrD, Disease)={ edge_index=[2, 1086] },
  (Gene, GNBR:J, Disease)={ edge_index=[2, 60468] },
  (Microbe, MIND:microbe-microbe:metabolic competition index, Microbe)={ edge_index=[2, 23562] },
  (Gene, GNBR:Y, Disease)={ edge_index=[2, 3896] },
  (Disease, Hetionet:DuG, Gene)={ edge_index=[2, 15462] },
  (Disease, Hetionet:DdG, Gene)={ edge_index=[2, 15246] },
  (Gene, GNBR:Md, Disease)={ edge_index

In [ ]:
import torch_geometric.transforms as T

data = T.ToUndirected()(data)

In [ ]:
# Conver From Homogeneous

homogeneous_data = data.to_homogeneous()
homogeneous_data

# Heterogeneous GNN

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


model = GNN(hidden_channels=64, out_channels=2)
model = to_hetero(model, data.metadata(), aggr='sum')

# MetaPath2Vec

In [ ]:
data

In [ ]:
data.metadata()[1]

In [ ]:
import torch

from torch_geometric.nn import MetaPath2Vec


metapath = [
    ('Microbe', 'HMDB:microbe-metabolite', 'Metabolite'),
    ('Metabolite', 'HMDB:metabolite-disease', 'Disease'),
    ('Disease','rev_HMDB:metabolite-disease', 'Metabolite'),
    ('Metabolite', 'rev_HMDB:microbe-metabolite', 'Microbe'),
]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MetaPath2Vec(data.edge_index_dict,
                     embedding_dim=32,
                     metapath=metapath,
                     walk_length=50,
                     context_size=7,
                     walks_per_node=5,
                     num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True)

In [ ]:
next(iter(loader))

In [ ]:
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
log_steps = 100
eval_steps = 2000
epoch = 1

model.train()

total_loss = 0
for i, (pos_rw, neg_rw) in enumerate(loader):
    optimizer.zero_grad()
    loss = model.loss(pos_rw.to(device), neg_rw.to(device))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    if (i + 1) % log_steps == 0:
        print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                f'Loss: {total_loss / log_steps:.4f}'))
        total_loss = 0

In [ ]:
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train(epoch, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('author', batch=data['author'].y_index.to(device))
    y = data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)


    for epoch in range(1, 6):
        train(epoch)
        #acc = test()
        print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
X = X.to(device)
y = y.to(device)

In [ ]:
train_ratio = 0.7
batch_size = 32

dataset = TensorDataset(X, y)

total_samples = len(dataset)
train_samples = int(train_ratio * total_samples)
test_samples = total_samples - train_samples

train_dataset, test_dataset = random_split(dataset, [train_samples, test_samples])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))

Train dataset size: 628
Test dataset size: 270


# Paper Dataset

In [ ]:
dataset = OGB_MAG(root='./data', preprocess='metapath2vec', transform=T.ToUndirected())
data_test = dataset[0]

In [ ]:
homo_test = data_test.to_homogeneous()

In [ ]:
homo_test

Data(edge_index=[2, 42182144], x=[1939743, 128], year=[1939743], y=[1939743], train_mask=[1939743], val_mask=[1939743], test_mask=[1939743], node_type=[1939743], edge_type=[42182144])

In [ ]:
from torch_geometric.nn import Node2Vec

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(
    homo_test.edge_index,
    embedding_dim=32,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

In [ ]:
loader = model.loader(batch_size=128, shuffle=True, num_workers=2)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
train()

1.3818033496211048

In [ ]:
model().shape

torch.Size([1939743, 32])

In [ ]:
model()[:2]

tensor([[ 2.1297, -2.0902, -0.9451,  1.7735,  0.4530, -0.4484,  0.5225,  0.0939,
         -0.1960, -1.3418,  0.1510,  0.5600,  0.5331,  0.1440,  1.0851, -0.6164,
         -0.0482,  0.1620, -0.4720,  0.8477,  0.0908, -1.5617, -0.2535, -0.9266,
          0.6317, -1.1368, -0.2709, -0.9450, -1.0220, -0.1377,  0.5472,  0.6550],
        [-0.0147, -1.6711, -1.4596,  0.1431, -1.4963,  1.0022, -1.5662,  0.9430,
          2.3672,  1.0997,  0.6574,  0.3336, -0.9135,  0.1145,  0.2533, -0.0840,
         -1.0511, -0.0825, -0.2376, -1.2604, -0.0584, -0.0909,  0.4628, -0.7270,
          0.5638,  1.2640, -1.5365, -0.1794,  0.3118,  1.1963,  0.4981, -1.1483]],
       device='cuda:0', grad_fn=<SliceBackward0>)